## Random Forest Hyperparameter Tuning (F1-Score)

In [4]:
# Load Libraries

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score

from sklearn.ensemble import RandomForestClassifier

import optuna

In [5]:
# Load Data

df = pd.read_csv('bank_4.csv', index_col=0)

In [6]:
# Train / Test Split

X = df.drop(columns=['churn', 'complain', 'umap_1', 'umap_2'])
y = df['churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [7]:
# Hyperparameter tuning

def objective(trial):
    
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000),
        'max_features': trial.suggest_int('max_features', 4, 53),
        'max_depth': trial.suggest_int('max_depth', 2, 50),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 32),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 32)
    }

    model = RandomForestClassifier(
        **params,
        random_state=42,
        n_jobs=-1)
    
    threshold = 0.34
    
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    scores = []
    
    for tr, te in skf.split(X_train, y_train):
        
        X_tr, X_te = X_train.iloc[tr], X_train.iloc[te]
        y_tr, y_te = y_train.iloc[tr], y_train.iloc[te]
        
        model.fit(X_tr, y_tr)
        prob = model.predict_proba(X_te)[:, 1]
        y_pred = np.where(prob < threshold, 0, 1)
        
        scores.append(f1_score(y_te, y_pred))
        
    return np.mean(scores)

In [8]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, show_progress_bar=True)

[I 2024-07-09 22:46:32,374] A new study created in memory with name: no-name-57b4f9c1-fb6c-4e79-abc8-72ad457831c4


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2024-07-09 22:46:41,811] Trial 0 finished with value: 0.6190332227272812 and parameters: {'n_estimators': 741, 'max_features': 12, 'max_depth': 34, 'min_samples_split': 9, 'min_samples_leaf': 16}. Best is trial 0 with value: 0.6190332227272812.
[I 2024-07-09 22:47:08,186] Trial 1 finished with value: 0.6164309027765146 and parameters: {'n_estimators': 1832, 'max_features': 37, 'max_depth': 46, 'min_samples_split': 29, 'min_samples_leaf': 14}. Best is trial 0 with value: 0.6190332227272812.
[I 2024-07-09 22:47:16,041] Trial 2 finished with value: 0.6171337233737887 and parameters: {'n_estimators': 555, 'max_features': 19, 'max_depth': 46, 'min_samples_split': 8, 'min_samples_leaf': 17}. Best is trial 0 with value: 0.6190332227272812.
[I 2024-07-09 22:47:39,626] Trial 3 finished with value: 0.6176841168328293 and parameters: {'n_estimators': 1715, 'max_features': 30, 'max_depth': 9, 'min_samples_split': 29, 'min_samples_leaf': 11}. Best is trial 0 with value: 0.6190332227272812.
[I 20

In [22]:
print("Best trial:", study.best_trial)
print("Best hyperparameters:", study.best_params)

Best trial: FrozenTrial(number=48, state=1, values=[0.6205821476984937], datetime_start=datetime.datetime(2024, 7, 9, 22, 55, 16, 239302), datetime_complete=datetime.datetime(2024, 7, 9, 22, 55, 22, 777301), params={'n_estimators': 494, 'max_features': 15, 'max_depth': 40, 'min_samples_split': 25, 'min_samples_leaf': 19}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=2000, log=False, low=100, step=1), 'max_features': IntDistribution(high=53, log=False, low=4, step=1), 'max_depth': IntDistribution(high=50, log=False, low=2, step=1), 'min_samples_split': IntDistribution(high=32, log=False, low=2, step=1), 'min_samples_leaf': IntDistribution(high=32, log=False, low=1, step=1)}, trial_id=48, value=None)
Best hyperparameters: {'n_estimators': 494, 'max_features': 15, 'max_depth': 40, 'min_samples_split': 25, 'min_samples_leaf': 19}
